In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install symspellpy

In [ ]:
import os 
import pandas as pd
import numpy as np
import datetime
from gensim import corpora
import gensim
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

import keras
from keras.layers import Input, Bidirectional, LSTM, Dense, RepeatVector, Concatenate, Activation, Lambda, Dot, Softmax, TimeDistributed, Dropout, Layer
from keras.models import Model
from keras import backend as K
from sklearn.model_selection import train_test_split
import warnings
from nltk.tokenize import word_tokenize
warnings.filterwarnings('ignore')
import nltk
nltk.download('punkt')

from collections import Counter
from sklearn.metrics import silhouette_score
# import umap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import os
import pkg_resources
from symspellpy import SymSpell, Verbosity
import nltk
nltk.download('averaged_perceptron_tagger')

In [5]:
from google.colab import drive

""" connect google drive """
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
  pass
else:
  sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# **Data Preprocess**

In [7]:
# selecting nouns
def f_noun(w_list):
  """
  :param w_list: word list to be processed
  :return: w_list with only nouns selected
  """
  return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] in ['NN','NNS','NNP','NNPS']]

In [8]:
# typo correction
def f_typo(w_list):
  """
  :param w_list: word list to be processed
  :return: w_list with typo fixed by symspell. words with no match up will be dropped
  """
  w_list_fixed = []
  for word in w_list:
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
    if suggestions:
      w_list_fixed.append(suggestions[0].term)
    else:
      pass
      # do word segmentation, deprecated for inefficiency
      # w_seg = sym_spell.word_segmentation(phrase=word)
      # w_list_fixed.extend(w_seg.corrected_string.split())
  return w_list_fixed

In [9]:
def preprocess_sent(review_list):
  tmp = False
  new_array = []
  new_review_list = []
  for sentence in review_list:
    sentence = sentence.lstrip().rstrip()
    sentence = sentence.split()
    if len(sentence) == 1:
      word = sentence[0]
      tmp = True
      new_array.append(word)
    else:
      if tmp == True:
        lastword = " ".join(new_array)
        word = lastword+" "+ " ".join(sentence)
        new_review_list.append(word)
        tmp = False
        new_array = []
      else:
        word = " ".join(sentence)
        new_review_list.append(word)
  return new_review_list

In [10]:
def preprocess_word(s):
  if not s:
    return None
  w_list = word_tokenize(s)
  # w_list = f_punct(w_list)
  w_list = f_noun(w_list)
  w_list = f_typo(w_list)
  # w_list = f_stem(w_list)
  # w_list = f_stopw(w_list)

  return w_list

In [28]:
def preprocess(docs, samp_size=None):
  if not samp_size:
    samp_size = 100
  n_docs = len(docs)
  sentences = []  # sentence level preprocessed
  token_lists = []  # word level preprocessed
  idx_in = []  # index of sample selected
  samp = np.random.choice(n_docs, samp_size)

  for i, idx in enumerate(samp):
    review = docs[idx].replace("[","").replace("]","").replace("'","").split(",")
    new_review = preprocess_sent(review)
    for sentence in new_review:
      token_list = preprocess_word(sentence)
      if token_list:
        idx_in.append(idx)
        sentences.append(sentence)
        token_lists.append(token_list)
    print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
  print('Preprocessing raw texts. Done!')
  return sentences, token_lists, idx_in

  print('Preprocessing raw texts ...')

In [12]:
# path = "/content/gdrive/MyDrive/fucking_paper/dataset/phone_segement.csv"
# path = "/content/gdrive/MyDrive/fucking_paper/dataset/segement.csv"
path = "/content/gdrive/MyDrive/fucking_paper/dataset/dataset_split/phone_train_pos.csv"

In [29]:
cwd = os.getcwd() 
with open(path, 'r', encoding="utf-8",errors='ignore') as file:
  meta = pd.read_csv(file)
# print(meta.shape) #(24130, 16)
# rws = meta.segement
rws = meta.segmentation 
samp_size = 51000

sentences = []  # sentence level preprocessed
token_lists = []  # word level preprocessed
idx_in = []  # index of sample selected
total_review_count = []

sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)

Preprocessing raw texts. Done!


In [31]:
print(sentences[0:15])

['second love', 'mobile comet high', 'end much', 'blemish free performs like new', 'happy condition ability arrive', 'nice good feature best', 'nice good price one little', 'scratch screen nothing major', 'battery seem', 'run quicker would like', 'love personally love product', 'upgrade so', 'fast charge still new', 'gen phone', 'honestly love prefer']


In [33]:
print(token_lists[0:15])

[['love'], ['comet'], ['end'], ['performs'], ['condition', 'ability'], ['feature'], ['price'], ['scratch', 'screen', 'nothing'], ['battery'], ['run'], ['product'], ['upgrade'], ['charge'], ['gen', 'phone'], ['prefer']]


# **Untils**

In [43]:
# def get_topic_words(token_lists, labels, topk, save_file, k=None):
def get_topic_words(token_lists, labels, k=None):
  """
  get top words within each topic from clustering results
  """
  if k is None:
    k = len(np.unique(labels))
  topics = ['' for _ in range(k)]
  for i, c in enumerate(token_lists):
    topics[labels[i]] += (' ' + ' '.join(c))
  word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
  # get sorted word counts
  word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
  # get topics
  topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

  return topics

def get_coherence(model, token_lists, measure='c_v'):
  """
  Get model coherence from gensim.models.coherencemodel
  :param model: Topic_Model object
  :param token_lists: token lists of docs
  :param topics: topics as top words
  :param measure: coherence metrics
  :return: coherence score
  """
  if model.method == 'LDA':
    cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary, coherence=measure)
  else:
    topics = get_topic_words(token_lists, model.cluster_model.labels_)
    cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary, coherence=measure)
    # print(cm.get_coherence_per_topic())
  return cm.get_coherence()

def get_silhouette(model):

  if model.method == 'LDA':
    return
  lbs = model.cluster_model.labels_
  vec = model.vec[model.method]
  return silhouette_score(vec, lbs)

def get_wordcloud(model, token_lists, topic):
  """
  :param model: Topic_Model object
  """
  if model.method == 'LDA':
    return
  print('Getting wordcloud for topic {} ...'.format(topic))
  lbs = model.cluster_model.labels_
  tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

  wordcloud = WordCloud(width=800, height=560,
              background_color='white', collocations=False,
              min_font_size=10).generate(tokens)

  # plot the WordCloud image
  plt.figure(figsize=(8, 5.6), facecolor=None)
  plt.imshow(wordcloud)
  print()
  plt.axis("off")
  plt.tight_layout(pad=0)

  print(tokens[0:10])
  print()

def assign_topic_todoc(model):
  if model.method == 'LDA':
    topic_vectors = []
    for doc_bow in model.corpus:
      doc_topics = model.ldamodel.get_document_topics(doc_bow)
      topic_vector = [0] * model.k
      for topic in doc_topics:
        topic_vector[topic[0]] = topic[1]
      topic_vectors.append(topic_vector)
    maxres=[np.argmax(x) for x in topic_vectors]
    return maxres
  else:
    lbs = model.cluster_model.labels_
    return lbs

# **AutoEncoder**

In [35]:
from keras.models import Sequential

In [36]:
class Autoencoder:
  """Autoencoder for learning latent space representation"""

  def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
    self.latent_dim = latent_dim
    self.activation = activation
    self.epochs = epochs
    self.batch_size = batch_size
    self.autoencoder = None
    self.encoder = None
    self.decoder = None
    self.his = None

  def _compile(self, input_dim):
    """
    compile the computational graph
    """
    input_vec = Input(shape=(input_dim,))
    encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
    # print("encoded: ",encoded)
    decoded = Dense(input_dim, activation=self.activation)(encoded)
    # print("decoded: ",decoded)
    self.autoencoder = Model(input_vec, decoded, name='autoencoder')
    # print(self.autoencoder.summary())
    self.encoder = Model(input_vec, encoded, name='encoder')
    # print()
    # print(self.encoder.summary())
    encoded_input = Input(shape=(self.latent_dim,))
    decoder_layer = self.autoencoder.layers[-1]
    self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input), name='decoder')
    # print()
    # print(self.decoder.summary())
    self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

  def fit(self, X):
    if not self.autoencoder:
      # print("input_dim形狀: ", X.shape)
      self._compile(X.shape[1])
    X_train, X_test = train_test_split(X)
    self.his = self.autoencoder.fit(X_train, X_train, epochs=200,
                     batch_size=128, shuffle=True,
                     validation_data=(X_test, X_test), verbose=0)

# **NTM**

In [37]:
class Sampling(Layer):
  def __init__(self, latent_dim):
    super(Sampling, self).__init__()
    self.latent_dim = latent_dim

  def call(self, inputs):
    mu, log_var = inputs
    epsilon = K.random_normal(shape=(K.shape(mu)[0], self.latent_dim))
    return mu + K.exp(0.5 * log_var) * epsilon

In [38]:
class NTM_Autoencoder:
  def __init__(self, latent_dim=32, intermediate_dim = 96, activation='relu', epochs=200, batch_size=128, num_topics=15, dropout = 0.2):
    self.latent_dim = latent_dim
    self.intermediate_dim = intermediate_dim
    self.activation = activation
    self.epochs = epochs
    self.batch_size = batch_size
    self.dropout = dropout
    self.autoencoder = None
    self.encoder = None
    self.decoder = None
    self.his = None
    # self.num_topics = num_topics

  def _compile(self, input_dim):
    """
    compile the computational graph
    """

    input_vec = Input(shape=(input_dim,))
    encoded = Dense(self.intermediate_dim, activation=self.activation)(input_vec)
    encoded = Dropout(self.dropout)(encoded)
    mu = Dense(self.latent_dim)(encoded)
    log_var = Dense(self.latent_dim)(encoded)

    def sampling(args):
      z_mean, z_log_var = args
      epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.latent_dim), mean=0., stddev=1.)
      return z_mean + K.exp(z_log_var / 2) * epsilon

    # z = Sampling(self.latent_dim)([mu, log_var])
    z = Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = Input(shape=(self.latent_dim,))
    x = Dense(self.intermediate_dim, activation=self.activation)(decoder_input)
    outputs = Dense(input_dim, activation=self.activation)(x)
    
    # decoded = Dense(self.latent_dim, activation=self.activation)(z)
    # outputs = Dropout(self.dropout)(decoded)
    
    self.encoder = Model(input_vec, [mu, log_var, z], name='encoder')
    # print(self.encoder.summary())
    # print()
    self.decoder = Model(decoder_input, outputs, name='decoder')
    # print(self.decoder.summary())
    # print()
    outputs = self.decoder(self.encoder(input_vec)[2])
    self.autoencoder = Model(input_vec, outputs, name='autoencoder')
    # print(self.autoencoder.summary())
    # print()
      
    def vae_loss(inputs, outputs):
      reconstruction_loss = K.mean(keras.losses.binary_crossentropy(inputs, outputs))
      kl_loss = -0.5 * K.sum(1 + log_var - K.square(mu) - K.exp(log_var))
      return K.mean(reconstruction_loss + kl_loss)

    # self.autoencoder.compile(optimizer='adam', loss=vae_loss)
    self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)
  
  def predict(self, x_test):
    return self.vae.predict(x_test)

  def fit(self, X):
    if not self.autoencoder:
      # print("input_dim形狀: ", X.shape)
      self._compile(X.shape[1])
    X_train, X_test = train_test_split(X)
    self.his = self.autoencoder.fit(X_train, X_train, epochs=200,
                     batch_size=128, shuffle=True,
                     validation_data=(X_test, X_test), verbose=0)

# **Topic Model**

In [39]:
# define model object
class Topic_Model:
  def __init__(self, token_lists, sentences, k=10, method='TFIDF'):
    """
    :param k: number of topics
    :param method: method chosen for the topic model
    """
    if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
      raise Exception('Invalid method!')
    self.k = k
    self.dictionary = None
    self.corpus = None
    # self.stopwords = None
    self.cluster_model = None
    self.ldamodel = None
    self.vec = {}
    self.gamma = 10  # parameter for reletive importance of lda
    self.method = method
    self.AE = None
    self.id = method + '_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    self.token_lists = token_lists
    # turn tokenized documents into a id <-> term dictionary
    self.dictionary = corpora.Dictionary(token_lists)
    # convert tokenized documents into a document-term matrix
    self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]
    self.topic_word_counts = None
    self.topics_ans = None

  def vectorize(self, sentences, token_lists, training=True, method=None):
    """
    Get vecotr representations from selected methods
    """
    # Default method
    if method is None:
      method = self.method

    # # turn tokenized documents into a id <-> term dictionary
    # self.dictionary = corpora.Dictionary(token_lists)
    # # convert tokenized documents into a document-term matrix
    # self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

    if training:
      # turn tokenized documents into a id <-> term dictionary
      dictionary = self.dictionary
      # convert tokenized documents into a document-term matrix
      corpus = [self.dictionary.doc2bow(text) for text in token_lists]
    else:
      dictionary = corpora.Dictionary(token_lists)
      corpus = [dictionary.doc2bow(text) for text in token_lists]

    if method == 'TFIDF':
      print('Getting vector representations for TF-IDF ...')
      tfidf = TfidfVectorizer()
      vec = tfidf.fit_transform(sentences)
      print('Getting vector representations for TF-IDF. Done!')
      return vec

    elif method == 'LDA':
      print('Getting vector representations for LDA ...')
      if not self.ldamodel:
        # self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, 
        #                         id2word=self.dictionary, passes=20)
        self.ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=self.k, 
                                 id2word=dictionary, passes=20,
                                 iterations=100)

      def get_vec_lda(model, corpus, k):
        """
        Get the LDA vector representation (probabilistic topic assignments for all documents)
        :return: vec_lda with dimension: (n_doc * n_topic)
        """
        n_doc = len(corpus)
        vec_lda = np.zeros((n_doc, k))
        for i in range(n_doc):
          # get the distribution for the i-th document in corpus
          for topic, prob in model.get_document_topics(corpus[i]):
            vec_lda[i, topic] = prob

        return vec_lda

      vec = get_vec_lda(self.ldamodel, corpus, self.k)
      print('Getting vector representations for LDA. Done!')
      return vec

    elif method == 'BERT':
      print('Getting vector representations for BERT ...')
      from sentence_transformers import SentenceTransformer
      model = SentenceTransformer('bert-base-nli-max-tokens')
      vec = np.array(model.encode(sentences, show_progress_bar=True))
      print('Getting vector representations for BERT. Done!')
      return vec

            
    elif method == 'LDA_BERT':
    #else:
      vec_lda = self.vectorize(sentences, token_lists, method='LDA')
      # print("vec_lda: ",vec_lda.shape) # (35116, 15)
      vec_bert = self.vectorize(sentences, token_lists, method='BERT')
      # print("vec_bert: ",vec_bert.shape) # (35116, 768)
      vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
      self.vec['LDA_BERT_FULL'] = vec_ldabert #[35116,783]
      if not self.AE:
        # self.AE = Autoencoder()
        self.AE = NTM_Autoencoder()
        print('Fitting Autoencoder ...')
        self.AE.fit(vec_ldabert)
        print('Fitting Autoencoder Done!')
      # vec = self.AE.encoder.predict(vec_ldabert)
      # return vec
      z_mean, _, _ = self.AE.encoder.predict(vec_ldabert)
      return z_mean

  def fit(self, sentences, token_lists, method=None, m_clustering=None):
    """
    Fit the topic model for selected method given the preprocessed data
    :docs: list of documents, each doc is preprocessed as tokens
    :return:
    """
    # Default method
    if method is None:
      method = self.method
    # Default clustering method
    if m_clustering is None:
      m_clustering = KMeans

    # turn tokenized documents into a id <-> term dictionary
    if not self.dictionary:
      self.dictionary = corpora.Dictionary(token_lists)
      # convert tokenized documents into a document-term matrix
      self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

    ######################################################
    ## Getting ldamodel or vector representations ##
    ######################################################

    if method == 'LDA':
      if not self.ldamodel:
        print('Fitting LDA ...')
        self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, 
                                 id2word=self.dictionary, passes=20)
        print('Fitting LDA Done!')
    else:
      print('Clustering embeddings ...')
      self.cluster_model = m_clustering(self.k)
      self.vec[method] = self.vectorize(sentences, token_lists, method)

      self.cluster_model.fit(self.vec[method])
      print('Clustering embeddings. Done!')
  
  def all_topic_word(self):
    if self.k is None:
      k = len(np.unique(self.cluster_model.labels_))
      self.k = k
    topics = ['' for _ in range(self.k)]
    for i, c in enumerate(self.token_lists):
      topics[self.cluster_model.labels_[i]] += (' ' + ' '.join(c))
    topic_word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    self.topic_word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), topic_word_counts))

  def save_all_topic_word(self):
    if self.k is None:
      k = len(np.unique(self.cluster_model.labels_))
      self.k = k
    topics = ['' for _ in range(self.k)]
    for i, c in enumerate(self.token_lists):
      topics[self.cluster_model.labels_[i]] += (' ' + ' '.join(c))
    topic_word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    topic_word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), topic_word_counts))
    return topic_word_counts
  
  # def print_thisTopic_topn(self, topicnum, topn, token_lists = None):
  def print_thisTopic_topn(self, topicnum, topn):
    # if token_lists == None:
    #   token_lists = self.token_lists
    
    if self.method == 'LDA':
      print('Print LDA Topic_topn...')
      topics = self.ldamodel.print_topic(topicnum, topn)
      print('Print LDA Topic_topn Done!')
      return topics
    else:
      if self.topics_ans == None:
        if self.topic_word_counts == None:
          self.all_topic_word()
        topics = list(map(lambda x: list(map(lambda x: x[0], x[:topn])), self.topic_word_counts))
        self.topics_ans = topics
        return topics[topicnum]
      else:
        return self.topics_ans[topicnum]
      
      # from sklearn.metrics.pairwise import euclidean_distances
      # # Get the data points that belong to the second cluster
      # cluster_words = np.array(self.token_lists)[self.cluster_model.labels_ == topicnum]
      # cluster_values = self.vec.get(self.method, None)[self.cluster_model.labels_ == topicnum]

      # # Get the distances between each point and the centroid of the cluster
      # distances = euclidean_distances(cluster_values, [self.cluster_model.cluster_centers_[1]])

      # # Get the indices of the top 5 closest points to the centroid
      # top_n_indices = np.argsort(distances, axis=0)[:topn]

      # # Print the top 5 values in the second cluster
      # print("Top BERT values in specific cluster:")
      # topics = []
      # for i in range(len(top_n_indices)):
      #     topics.append(cluster_words[top_n_indices[i][0]])
      #     # topics.append(cluster_values[top_n_indices[i][0]])
      # return topics

  def predict(self, sentences, token_lists, out_of_sample=None):
    """
    Predict topics for new_documents
    """
    # Default as False
    out_of_sample = out_of_sample is not None

    if out_of_sample:
      corpus = [self.dictionary.doc2bow(text) for text in token_lists]
      if self.method != 'LDA':
        vec = self.vectorize(sentences, token_lists, training=False)
        print(vec)
    else:
      corpus = self.corpus
      vec = self.vec.get(self.method, None)

    if self.method == "LDA":
      lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                            key=lambda x: x[1], reverse=True)[0][0], corpus)))
    else:
      lbs = self.cluster_model.predict(vec)
      
    return lbs

  
  # def get_each_doc_topics(self, doc_bow):
  #   if self.method == 'LDA':
  #     print('Get doc LDA topic...')
  #     self.ldamodel.get_document_topics(doc_bow, minimum_probability=0.03)
  #     print('Get doc LDA topic Done!')
  #   else:
  #     lbs = self.cluster_model.labels_
  #     return lbs

# **Train**

In [40]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [15]:
# path = "/content/gdrive/MyDrive/fucking_paper/dataset/phone_segement.csv"
# path = "/content/gdrive/MyDrive/fucking_paper/dataset/segement.csv"
path = "/content/gdrive/MyDrive/fucking_paper/dataset/dataset_split/phone_train_pos.csv"

In [41]:
# def train_Topic_model(gamma = 15, ntopic = 10, method = "BERT"):
""" Hyper parameter """
gamma = 12
ntopic = 10
# method = "LDA_BERT"
method = "BERT"
# samp_size = 51000

# cwd = os.getcwd() 
# with open(path, 'r', encoding="utf-8",errors='ignore') as file:
#   meta = pd.read_csv(file)
# # print(meta.shape) #(24130, 16)
# # rws = meta.segement
# rws = meta.segmentation 

# sentences = []  # sentence level preprocessed
# token_lists = []  # word level preprocessed
# idx_in = []  # index of sample selected
# total_review_count = []

# sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)
# for i, review in enumerate(rws):
#   review = review.replace("[","").replace("]","").replace("'","").split(",")
#   new_review = sentence_preprocess(review)
#   count = len(new_review)
#   total_review_count.append(count)
#   for sentence in new_review:
#     sentences.append(sentence.lstrip().rstrip())
#     # print(sentence.lstrip().rstrip())
#     token_list = word_tokenize(sentence.lstrip().rstrip())
#     token_lists.append(token_list)

# print("token len: ",len(token_lists)) # 35116個句子 # 275961個句子 是list
# print("sentences len: ",len(sentences))  # 35116個句子 # 275961個句子
# print(token_lists)  # [ [w1,w2,w3..], [w1,w2], ...]
# print(sentences) # [[w1 w2 w3 ..], [w50 w52], ....]

tm = Topic_Model(token_lists, sentences, k = ntopic, method = method)
# Fit the topic model by chosen method
tm.fit(sentences, token_lists)

  # return tm

Clustering embeddings ...
Getting vector representations for BERT ...


Batches:   0%|          | 0/8381 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
Clustering embeddings. Done!


In [46]:
for i in range(tm.k):
  print(tm.print_thisTopic_topn(i, 10))

['battery', 'use', 'nothing', 'screen', 'camera', 'work', 'time', 'lack', 'day', 'price']
['price', 'screen', 'battery', 'fast', 'work', 'everything', 'use', 'lot', 'charge', 'camera']
['iphone', 'android', 'smartphone', 'phone', 'i', 'use', 'phones', 'is', 'smartphones', 'of']
['use', 'i', 'time', 'get', 'price', 'screen', 'work', 'feature', 'camera', 'look']
['love', 'use', 'work', 'quality', 'price', 'brand', 'look', 'i', 'thing', 'feature']
['battery', 'work', 'life', 'camera', 'use', 'love', 'phone', 'screen', 'everything', 'quality']
['battery', 'phone', 'card', 'life', 'use', 'sim', 'screen', 'charger', 'camera', 'fingerprint']
['problem', 'battery', 'scratch', 'screen', 'thing', 'use', 'drop', 'work', 'get', 'camera']
['work', 'love', 'quality', 'camera', 'product', 'use', 'condition', 'screen', 'excellent', 'look']
['day', 'year', 'battery', 'month', 'use', 'hour', 'time', 'week', 'work', 'charge']


In [ ]:
def evaluate(model):
  coherence_score = get_coherence(model, model.token_lists, 'c_v')
  Silhouette_score = get_silhouette(model)
  print('Coherence:', coherence_score)
  print('Silhouette Score:', )

  return coherence_score, Silhouette_score

In [ ]:
coherence_score = get_coherence(tm, tm.token_lists, 'c_v')
Silhouette_score = get_silhouette(tm)
print('Coherence:', coherence_score)
print('Silhouette Score:', )

[0.5494844273732945, 0.6530064163098263, 0.5708154304631625, 0.6699022382060907, 0.6699022382060907, 0.6578193746074016, 0.6669641905334281, 0.42094694429426066, 0.5479320610218991, 0.6641676252195557]
Coherence: 0.6070940946235011
Silhouette Score:


In [ ]:
# # topics = get_topic_words(model.token_lists, model.cluster_model.labels_)
#  # assign topics to each document
# # print([max(model.ldamodel.get_document_topics(doc), key=lambda x:x[1])[0] for doc in model.corpus])
# print(model.ldamodel.show_topics(num_topics=20, num_words=10, formatted=False))

[0, 8, 18, 16, 0, 11, 0, 3, 1, 1, 13, 12, 5, 18, 12, 4, 8, 0, 15, 12, 10, 3, 6, 14, 4, 18, 2, 3, 2, 11, 15, 9, 8, 9, 7, 7, 17, 11, 8, 15, 16, 13, 9, 15, 1, 3, 13, 11, 3, 17, 18, 12, 14, 2, 17, 3, 9, 8, 10, 0, 5, 9, 14, 7, 7, 1, 10, 3, 18, 0, 18, 17, 1, 15, 8, 9, 4, 13, 4, 4, 18, 9, 2, 2, 12, 6, 11, 3, 11, 18, 2, 3, 11, 9, 4, 12, 3, 1, 3, 8, 14, 11, 3, 18, 10, 3, 16, 5, 4, 2, 11, 16, 17, 10, 2, 5, 14, 6, 11, 3, 18, 16, 0, 10, 11, 5, 5, 13, 6, 7, 0, 17, 8, 15, 5, 15, 6, 7, 6, 16, 10, 6, 16, 14, 10, 18, 10, 15, 18, 2, 10, 0, 2, 18, 1, 11, 11, 1, 3, 0, 8, 11, 1, 3, 5, 7, 3, 10, 16, 13, 16, 4, 6, 7, 5, 8, 3, 15, 1, 12, 1, 14, 7, 18, 18, 1, 3, 2, 3, 2, 0, 2, 1, 14, 11, 12, 3, 17, 18, 14, 16, 13, 17, 19, 16, 0, 6, 5, 14, 2, 4, 13, 2, 8, 14, 16, 4, 2, 6, 14, 5, 7, 19, 5, 11, 0, 18, 18, 1, 11, 18, 0, 4, 4, 11, 2, 0, 3, 4, 4, 5, 8, 1, 0, 12, 3, 11, 18, 17, 0, 9, 12, 5, 10, 18, 17, 0, 8, 14, 9, 7, 4, 1, 12, 1, 9, 9, 6, 17, 18, 4, 13, 6, 12, 11, 10, 11, 3, 13, 14, 12, 10, 7, 4, 11, 19, 0, 1, 13, 6

In [ ]:
# # get top words for each topic
# topics = []
# for topic in tm.ldamodel.show_topics(num_topics=20, num_words=3, formatted=False):
#   print("Topic {}:".format(topic[0]))
#   tmp = []
#   for word, prob in topic[1]:
#     print("\t", word, prob)
#     tmp.append(word)
#   topics.append(tmp)

# print(topics)

AttributeError: ignored

In [ ]:
# print(topics)

[['use', 'apps', 'amazon', 'screen', 'go'], ['different', 'drop', 'problem', 'end', 'screen'], ['battery', 'great', 'good', 'life', 'camera'], ['little', 'not', 'low', 'less', 'small'], ['like', 'so', 'though', 'would', 'even'], ['battery', 'get', 'screen', 'use', 'bad'], ['not', 'use', 'battery', 'one', 'get'], ['phone', 'battery', 'phones', 'mobile', 'use'], ['screen', 'use', 'camera', 'device', 'like'], ['new', 'get', 'time', 'first', 'need'], ['easy', 'fine', 'free', 'perfect', 'use'], ['buy', 'price', 'purchase', 'pay', 'go'], ['enough', 'like', 'right', 'open', 'able'], ['fast', 'many', 'high', 'much', 'big'], ['android', 'iphone', 'use', 'smartphone', 'is'], ['great', 'good', 'love', 'happy', 'excellent'], ['month', 'day', 'year', 'hour', 'week'], ['better', 'good', 'love', 'great', 'get'], ['bad', 'not', 'wrong', 'terrible', 'problem'], ['not', 'never', 'without', 'nothing', 'can']]


In [ ]:
# print(topics[2])

['would', 'also', 'work', 'say', 'fine']


In [ ]:
# # coherence_score, Silhouette_score = evaluate(model)
# # print(coherence_score)
# # print(Silhouette_score)
# topic_dict = assign_topic_todoc(model)

In [ ]:
# # print(len(topic_dict))
# print(topic_dict)

[0, 8, 18, 16, 0, 11, 0, 3, 1, 1, 13, 13, 5, 18, 12, 4, 8, 0, 15, 12, 10, 3, 6, 14, 4, 18, 2, 3, 2, 11, 15, 9, 8, 9, 7, 7, 17, 11, 8, 15, 16, 13, 9, 15, 1, 3, 13, 11, 3, 17, 18, 12, 14, 2, 17, 3, 9, 8, 10, 0, 5, 9, 14, 10, 7, 1, 10, 3, 18, 0, 18, 17, 1, 15, 8, 9, 18, 13, 4, 4, 18, 9, 2, 1, 12, 6, 11, 3, 11, 18, 2, 3, 11, 9, 4, 12, 3, 1, 3, 8, 14, 11, 3, 18, 10, 3, 16, 5, 4, 2, 11, 16, 17, 10, 2, 5, 14, 6, 11, 3, 18, 16, 0, 10, 11, 5, 5, 13, 6, 7, 15, 17, 8, 15, 5, 15, 6, 7, 6, 16, 10, 18, 16, 14, 10, 18, 10, 15, 18, 2, 10, 0, 2, 18, 1, 11, 4, 1, 3, 0, 8, 11, 1, 3, 5, 7, 3, 10, 16, 13, 16, 4, 6, 7, 5, 8, 3, 15, 1, 12, 1, 14, 7, 18, 18, 1, 3, 2, 3, 2, 0, 2, 1, 14, 11, 12, 3, 17, 18, 14, 16, 13, 17, 19, 16, 0, 6, 5, 14, 2, 4, 13, 2, 8, 14, 16, 4, 2, 6, 14, 5, 7, 19, 5, 11, 0, 18, 18, 1, 11, 18, 0, 4, 4, 11, 2, 0, 3, 4, 4, 5, 8, 1, 0, 12, 3, 11, 10, 17, 0, 9, 12, 5, 10, 18, 17, 0, 8, 14, 9, 7, 4, 1, 12, 1, 9, 9, 6, 17, 18, 4, 13, 6, 12, 11, 10, 11, 3, 13, 14, 12, 10, 7, 4, 11, 19, 0, 1, 13

In [ ]:
# cwd = os.getcwd() 
# with open(path, 'r', encoding="utf-8",errors='ignore') as file:
#   meta = pd.read_csv(file)
# # print(meta.shape) #(24130, 16)
# rws = meta.segement

# sentences = []  # sentence level preprocessed
# token_lists = []  # word level preprocessed
# # idx_in = []  # index of sample selected
# total_review_count = []

# for i, review in enumerate(rws):
#   review = review.replace("[","").replace("]","").replace("'","").split(",")
#   new_review = sentence_preprocess(review)
#   count = len(new_review)
#   total_review_count.append(count)
#   for sentence in new_review:
#     sentences.append(sentence.lstrip().rstrip())
#     # print(sentence.lstrip().rstrip())
#     token_list = word_tokenize(sentence.lstrip().rstrip())
#     token_lists.append(token_list)

In [ ]:
# # topics = get_topic_words(model.token_lists, model.cluster_model.labels_)
# result = []
# result_word = []
# index = 0
# for count in total_review_count:
#   word = ""
#   tmp2 = topic_dict[index:index+count]
#   unique_topic = np.unique(np.array(tmp2))
#   for x in  unique_topic:
#     tmp = ' '.join(a for a in topics[x])
#     word = word+ tmp+ " "
#   # word = ' '.join([topics[x] for x in tmp2])
#   tmp3 = ', '.join([str(x) for x in tmp2])
#   index += count
#   result.append(tmp3)
#   result_word.append(word)
  
# # res=pd.DataFrame({"assign_topic":result})
# # res_2 =pd.DataFrame({"bert_reviewwords":result_word})
# # resdf=pd.concat([meta,res,res_2], axis=1)
# # resdf.to_csv('/content/gdrive/MyDrive/fucking_paper/dataset/topic_phone_tmp.csv',index=None)

In [ ]:
# res=pd.DataFrame({"assign_topic":result})

In [ ]:
# print(res)

                                            assign_topic
0      13, 8, 5, 4, 4, 4, 5, 0, 3, 13, 8, 9, 9, 4, 19...
1                                 7, 5, 10, 16, 0, 1, 18
2      5, 19, 0, 0, 7, 11, 13, 5, 4, 9, 10, 18, 4, 0,...
3                   14, 4, 6, 17, 12, 17, 4, 18, 3, 4, 8
4                          5, 11, 14, 4, 3, 16, 7, 10, 7
...                                                  ...
23119  17, 16, 5, 5, 14, 0, 4, 2, 8, 15, 16, 7, 8, 9,...
23120  4, 13, 1, 4, 7, 19, 13, 6, 7, 10, 1, 6, 4, 3, ...
23121  13, 1, 14, 15, 5, 10, 5, 14, 5, 18, 8, 15, 14,...
23122                        16, 16, 7, 4, 3, 14, 12, 11
23123                             7, 11, 16, 7, 1, 0, 12

[23124 rows x 1 columns]


In [ ]:
# res_2 =pd.DataFrame({"lda_reviewwords":result_word})

In [ ]:
# print(res_2)

                                         lda_reviewwords
0      charge say display new fast look love first ca...
1      charge say display love first case great works...
2      new fast look everything amazing way love firs...
3      charge say display everything amazing way love...
4      charge say display new fast look love first ca...
...                                                  ...
23119  charge say display new fast look everything am...
23120  charge say display new fast look everything am...
23121  charge say display new fast look everything am...
23122  charge say display new fast look everything am...
23123  everything amazing way love first case android...

[23124 rows x 1 columns]


In [ ]:
# resdf=pd.concat([meta,res,res_2], axis=1)

In [ ]:
# print(resdf)

In [ ]:
# resdf.to_csv('/content/gdrive/MyDrive/fucking_paper/dataset/0322_lda.csv',index=None)

In [ ]:
# for k in range(tm.k):
topics = ['' for _ in range(tm.k)]
for i, c in enumerate(token_lists):
  topics[tm.cluster_model.labels_[i]] += (' ' + ' '.join(c))
word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
# get sorted word counts
word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
# get topics
topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))
print(topics)

[['day', 'new', 'year', 'last', 'battery', 'use', 'buy', 'get', 'month', 'one'], ['good', 'new', 'like', 'it', 'love', 'great', 'use', 'get', 'brand', 'best'], ['without', 'little', 'small', 'low', 'less', 'never', 'use', 'cheap', 'battery', 'good'], ['great', 'good', 'work', 'love', 'battery', 'best', 'use', 'life', 'new', 'camera'], ['great', 'good', 'work', 'love', 'excellent', 'nice', 'really', 'happy', 'quality', 'camera'], ['it', 'use', 'get', 'one', 'like', 'go', 'new', 'i', 'come', 'time'], ['it', 'bad', 'get', 'problem', 'screen', 'like', 'battery', 'use', 'scratch', 'need'], ['battery', 'phone', 'card', 'sim', 'life', 'use', 'screen', 'apps', 'it', 'camera'], ['fast', 'great', 'big', 'price', 'screen', 'it', 'large', 'much', 'high', 'use'], ['iphone', 'android', 'smartphone', 'phone', 'use', 'i', 'it', 'good', 'ios', 'iphones']]


# Main

In [ ]:
def main():
  model = train_Topic_model()
  # coherence_score, Silhouette_score = evaluate(model)
  # print(coherence_score)
  # print(Silhouette_score)
  topic_dict = assign_topic_todoc(tm)
  topics = get_topic_words(model.token_lists, model.cluster_model.labels_)
  result = []
  result_word = []
  index = 0
  for count in total_review_count:
    tmp2 = topic_dict[index:index+count]
    word = ' '.join([topics[x] for x in tmp2])
    tmp3 = ', '.join([str(x) for x in tmp2])
    index += count
    result.append(tmp3)
    result_word.append(word)
    
  res=pd.DataFrame({"assign_topic":result})
  res_2 =pd.DataFrame({"bert_reviewwords":result_word})
  resdf=pd.concat([meta,res,res_2], axis=1)
  resdf.to_csv('/content/gdrive/MyDrive/fucking_paper/dataset/topic_phone_tmp.csv',index=None)


  # print(result)
  # print(resdf.columns)
  # print(len(topic_dict))
  # print(len(total_review_count))

In [ ]:
# if __name__ == '__main__':
#   main()